<a href="https://colab.research.google.com/github/damjam212/CV-NLP/blob/main/notebooks/preprocess_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Optional cell to install all dependencies

In [ ]:
! pip install youtube_transcript_api
! pip install pytubefix
! pip install moviepy

### All imports

In [ ]:
import cv2
from youtube_transcript_api import YouTubeTranscriptApi
from pytubefix import YouTube

import collections
import numpy as np

# nlp toolkit
import nltk

# cv model
import tensorflow as tf
import tensorflow_hub as hub


import imageio
from IPython.display import HTML, display
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches


from IPython.display import display, HTML,clear_output
from moviepy.editor import ImageSequenceClip

# creating dataset
import csv

### Download specific video from youtube for analysis

In [ ]:

#xwyPjhRoeNc
#nhoikoUEI8U
video_id = "nhoikoUEI8U"
subtitles = YouTubeTranscriptApi.get_transcript(video_id)
print(len(subtitles))

yt = YouTube(f"https://www.youtube.com/watch?v={video_id}")
stream = yt.streams.filter(res="720p").first()

#print(yt.streams.filter(res="720p").first())
destination_path = "../videos"

video_file = stream.download(output_path=destination_path)


# def cv2_to_mediapipe_image(cv2_image):
#     rgb_image = cv2.cvtColor(cv2_image, cv2.COLOR_BGR2RGB)
#     image = mp.solutions.mediapipe.python.solution_base.Image(
#         width=rgb_image.shape[1],
#         height=rgb_image.shape[0],
#         rgb_data=np.frombuffer(rgb_image.tobytes(), dtype=np.uint8)
#     )

#     return image

129


### Movnet in HPE

In [ ]:
model_name = "movenet_lightning" #@param ["movenet_lightning", "movenet_thunder", "movenet_lightning_f16.tflite", "movenet_thunder_f16.tflite", "movenet_lightning_int8.tflite", "movenet_thunder_int8.tflite"]

if "tflite" in model_name:
  if "movenet_lightning_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/float16/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite
    input_size = 256
  elif "movenet_lightning_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/int8/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/int8/4?lite-format=tflite
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_path="model.tflite")
  interpreter.allocate_tensors()

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    # TF Lite format expects tensor type of uint8.
    input_image = tf.cast(input_image, dtype=tf.uint8)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    # Invoke inference.
    interpreter.invoke()
    # Get the model prediction.
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    return keypoints_with_scores

else:
  if "movenet_lightning" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
    input_size = 192
  elif "movenet_thunder" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    model = module.signatures['serving_default']

    # SavedModel format expects tensor type of int32.
    input_image = tf.cast(input_image, dtype=tf.int32)
    # Run model inference.
    outputs = model(input_image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints_with_scores = outputs['output_0'].numpy()
    return keypoints_with_scores

# Define the edges between keypoints to draw the skeleton
KEYPOINT_EDGES = {
    (0, 1): 'm', (0, 2): 'c', (1, 3): 'y', (2, 4): 'y',
    (0, 5): 'm', (0, 6): 'c', (5, 7): 'm', (7, 9): 'y',
    (6, 8): 'c', (8, 10): 'y', (5, 6): 'c', (5, 11): 'm',
    (6, 12): 'c', (11, 12): 'y', (11, 13): 'm', (13, 15): 'y',
    (12, 14): 'c', (14, 16): 'y'
}

def draw_keypoints(frame, keypoints, confidence_threshold):
    """Draws the keypoints and the skeleton on the image."""
    y, x, _ = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    # Draw keypoints
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0, 255, 0), -1)

    # Draw skeleton
    for edge, color in KEYPOINT_EDGES.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]

        if c1 > confidence_threshold and c2 > confidence_threshold:
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)


#process_video(video_file)

In [ ]:
#@title Helper functions for visualization

# Dictionary that maps from joint names to keypoint indices.
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

# Maps bones to a matplotlib color name.
KEYPOINT_EDGE_INDS_TO_COLOR = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def _keypoints_and_edges_for_display(keypoints_with_scores,
                                     height,
                                     width,
                                     keypoint_threshold=0.11):
  """Returns high confidence keypoints and edges for visualization.

  Args:
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    height: height of the image in pixels.
    width: width of the image in pixels.
    keypoint_threshold: minimum confidence score for a keypoint to be
      visualized.

  Returns:
    A (keypoints_xy, edges_xy, edge_colors) containing:
      * the coordinates of all keypoints of all detected entities;
      * the coordinates of all skeleton edges of all detected entities;
      * the colors in which the edges should be plotted.
  """
  keypoints_all = []
  keypoint_edges_all = []
  edge_colors = []
  num_instances, _, _, _ = keypoints_with_scores.shape
  for idx in range(num_instances):
    kpts_x = keypoints_with_scores[0, idx, :, 1]
    kpts_y = keypoints_with_scores[0, idx, :, 0]
    kpts_scores = keypoints_with_scores[0, idx, :, 2]
    kpts_absolute_xy = np.stack(
        [width * np.array(kpts_x), height * np.array(kpts_y)], axis=-1)
    kpts_above_thresh_absolute = kpts_absolute_xy[
        kpts_scores > keypoint_threshold, :]
    keypoints_all.append(kpts_above_thresh_absolute)

    for edge_pair, color in KEYPOINT_EDGE_INDS_TO_COLOR.items():
      if (kpts_scores[edge_pair[0]] > keypoint_threshold and
          kpts_scores[edge_pair[1]] > keypoint_threshold):
        x_start = kpts_absolute_xy[edge_pair[0], 0]
        y_start = kpts_absolute_xy[edge_pair[0], 1]
        x_end = kpts_absolute_xy[edge_pair[1], 0]
        y_end = kpts_absolute_xy[edge_pair[1], 1]
        line_seg = np.array([[x_start, y_start], [x_end, y_end]])
        keypoint_edges_all.append(line_seg)
        edge_colors.append(color)
  if keypoints_all:
    keypoints_xy = np.concatenate(keypoints_all, axis=0)
  else:
    keypoints_xy = np.zeros((0, 17, 2))

  if keypoint_edges_all:
    edges_xy = np.stack(keypoint_edges_all, axis=0)
  else:
    edges_xy = np.zeros((0, 2, 2))
  return keypoints_xy, edges_xy, edge_colors


def draw_prediction_on_image(
    image, keypoints_with_scores, crop_region=None, close_figure=False,
    output_image_height=None):
  """Draws the keypoint predictions on image.

  Args:
    image: A numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    crop_region: A dictionary that defines the coordinates of the bounding box
      of the crop region in normalized coordinates (see the init_crop_region
      function below for more detail). If provided, this function will also
      draw the bounding box on the image.
    output_image_height: An integer indicating the height of the output image.
      Note that the image aspect ratio will be the same as the input image.

  Returns:
    A numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  # To remove the huge white borders
  fig.tight_layout(pad=0)
  ax.margins(0)
  ax.set_yticklabels([])
  ax.set_xticklabels([])
  plt.axis('off')

  im = ax.imshow(image)
  line_segments = LineCollection([], linewidths=(4), linestyle='solid')
  ax.add_collection(line_segments)
  # Turn off tick labels
  scat = ax.scatter([], [], s=60, color='#FF1493', zorder=3)

  (keypoint_locs, keypoint_edges,
   edge_colors) = _keypoints_and_edges_for_display(
       keypoints_with_scores, height, width)

  line_segments.set_segments(keypoint_edges)
  line_segments.set_color(edge_colors)
  if keypoint_edges.shape[0]:
    line_segments.set_segments(keypoint_edges)
    line_segments.set_color(edge_colors)
  if keypoint_locs.shape[0]:
    scat.set_offsets(keypoint_locs)

  if crop_region is not None:
    xmin = max(crop_region['x_min'] * width, 0.0)
    ymin = max(crop_region['y_min'] * height, 0.0)
    rec_width = min(crop_region['x_max'], 0.99) * width - xmin
    rec_height = min(crop_region['y_max'], 0.99) * height - ymin
    rect = patches.Rectangle(
        (xmin,ymin),rec_width,rec_height,
        linewidth=1,edgecolor='b',facecolor='none')
    ax.add_patch(rect)

  fig.canvas.draw()
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
  image_from_plot = image_from_plot.reshape(
      fig.canvas.get_width_height()[::-1] + (3,))
  plt.close(fig)
  if output_image_height is not None:
    output_image_width = int(output_image_height / height * width)
    image_from_plot = cv2.resize(
        image_from_plot, dsize=(output_image_width, output_image_height),
         interpolation=cv2.INTER_CUBIC)
  return image_from_plot

def to_gif(images, duration):
  """Converts image sequence (4D numpy array) to gif."""
  imageio.mimsave('./animation.gif', images, duration=duration)
  return embed.embed_file('./animation.gif')

def progress(value, max=100):
  return HTML("""

          {value}

  """.format(value=value, max=max))

In [ ]:
# model_name = "movenet_lightning" #@param ["movenet_lightning", "movenet_thunder", "movenet_lightning_f16.tflite", "movenet_thunder_f16.tflite", "movenet_lightning_int8.tflite", "movenet_thunder_int8.tflite"]

# if "tflite" in model_name:
#   if "movenet_lightning_f16" in model_name:
#     !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/float16/4?lite-format=tflite
#     input_size = 192
#   elif "movenet_thunder_f16" in model_name:
#     !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite
#     input_size = 256
#   elif "movenet_lightning_int8" in model_name:
#     !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/int8/4?lite-format=tflite
#     input_size = 192
#   elif "movenet_thunder_int8" in model_name:
#     !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/int8/4?lite-format=tflite
#     input_size = 256
#   else:
#     raise ValueError("Unsupported model name: %s" % model_name)

#   # Initialize the TFLite interpreter
#   interpreter = tf.lite.Interpreter(model_path="model.tflite")
#   interpreter.allocate_tensors()

#   def movenet(input_image):
#     """Runs detection on an input image.

#     Args:
#       input_image: A [1, height, width, 3] tensor represents the input image
#         pixels. Note that the height/width should already be resized and match the
#         expected input resolution of the model before passing into this function.

#     Returns:
#       A [1, 1, 17, 3] float numpy array representing the predicted keypoint
#       coordinates and scores.
#     """
#     # TF Lite format expects tensor type of uint8.
#     input_image = tf.cast(input_image, dtype=tf.uint8)
#     input_details = interpreter.get_input_details()
#     output_details = interpreter.get_output_details()
#     interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
#     # Invoke inference.
#     interpreter.invoke()
#     # Get the model prediction.
#     keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
#     return keypoints_with_scores

# else:
#   if "movenet_lightning" in model_name:
#     module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
#     input_size = 192
#   elif "movenet_thunder" in model_name:
#     module = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
#     input_size = 256
#   else:
#     raise ValueError("Unsupported model name: %s" % model_name)

#   def movenet(input_image):
#     """Runs detection on an input image.

#     Args:
#       input_image: A [1, height, width, 3] tensor represents the input image
#         pixels. Note that the height/width should already be resized and match the
#         expected input resolution of the model before passing into this function.

#     Returns:
#       A [1, 1, 17, 3] float numpy array representing the predicted keypoint
#       coordinates and scores.
#     """
#     model = module.signatures['serving_default']

#     # SavedModel format expects tensor type of int32.
#     input_image = tf.cast(input_image, dtype=tf.int32)
#     # Run model inference.
#     outputs = model(input_image)
#     # Output is a [1, 1, 17, 3] tensor.
#     keypoints_with_scores = outputs['output_0'].numpy()
#     return keypoints_with_scores

In [ ]:
#@title Cropping Algorithm

# Confidence score to determine whether a keypoint prediction is reliable.
MIN_CROP_KEYPOINT_SCORE = 0.2

def init_crop_region(image_height, image_width):
  """Defines the default crop region.

  The function provides the initial crop region (pads the full image from both
  sides to make it a square image) when the algorithm cannot reliably determine
  the crop region from the previous frame.
  """
  if image_width > image_height:
    box_height = image_width / image_height
    box_width = 1.0
    y_min = (image_height / 2 - image_width / 2) / image_height
    x_min = 0.0
  else:
    box_height = 1.0
    box_width = image_height / image_width
    y_min = 0.0
    x_min = (image_width / 2 - image_height / 2) / image_width

  return {
    'y_min': y_min,
    'x_min': x_min,
    'y_max': y_min + box_height,
    'x_max': x_min + box_width,
    'height': box_height,
    'width': box_width
  }

def torso_visible(keypoints):
  """Checks whether there are enough torso keypoints.

  This function checks whether the model is confident at predicting one of the
  shoulders/hips which is required to determine a good crop region.
  """
  return ((keypoints[0, 0, KEYPOINT_DICT['left_hip'], 2] >
           MIN_CROP_KEYPOINT_SCORE or
          keypoints[0, 0, KEYPOINT_DICT['right_hip'], 2] >
           MIN_CROP_KEYPOINT_SCORE) and
          (keypoints[0, 0, KEYPOINT_DICT['left_shoulder'], 2] >
           MIN_CROP_KEYPOINT_SCORE or
          keypoints[0, 0, KEYPOINT_DICT['right_shoulder'], 2] >
           MIN_CROP_KEYPOINT_SCORE))

def determine_torso_and_body_range(
    keypoints, target_keypoints, center_y, center_x):
  """Calculates the maximum distance from each keypoints to the center location.

  The function returns the maximum distances from the two sets of keypoints:
  full 17 keypoints and 4 torso keypoints. The returned information will be
  used to determine the crop size. See determineCropRegion for more detail.
  """
  torso_joints = ['left_shoulder', 'right_shoulder', 'left_hip', 'right_hip']
  max_torso_yrange = 0.0
  max_torso_xrange = 0.0
  for joint in torso_joints:
    dist_y = abs(center_y - target_keypoints[joint][0])
    dist_x = abs(center_x - target_keypoints[joint][1])
    if dist_y > max_torso_yrange:
      max_torso_yrange = dist_y
    if dist_x > max_torso_xrange:
      max_torso_xrange = dist_x

  max_body_yrange = 0.0
  max_body_xrange = 0.0
  for joint in KEYPOINT_DICT.keys():
    if keypoints[0, 0, KEYPOINT_DICT[joint], 2] < MIN_CROP_KEYPOINT_SCORE:
      continue
    dist_y = abs(center_y - target_keypoints[joint][0]);
    dist_x = abs(center_x - target_keypoints[joint][1]);
    if dist_y > max_body_yrange:
      max_body_yrange = dist_y

    if dist_x > max_body_xrange:
      max_body_xrange = dist_x

  return [max_torso_yrange, max_torso_xrange, max_body_yrange, max_body_xrange]

def determine_crop_region(
      keypoints, image_height,
      image_width):
  """Determines the region to crop the image for the model to run inference on.

  The algorithm uses the detected joints from the previous frame to estimate
  the square region that encloses the full body of the target person and
  centers at the midpoint of two hip joints. The crop size is determined by
  the distances between each joints and the center point.
  When the model is not confident with the four torso joint predictions, the
  function returns a default crop which is the full image padded to square.
  """
  target_keypoints = {}
  for joint in KEYPOINT_DICT.keys():
    target_keypoints[joint] = [
      keypoints[0, 0, KEYPOINT_DICT[joint], 0] * image_height,
      keypoints[0, 0, KEYPOINT_DICT[joint], 1] * image_width
    ]

  if torso_visible(keypoints):
    center_y = (target_keypoints['left_hip'][0] +
                target_keypoints['right_hip'][0]) / 2;
    center_x = (target_keypoints['left_hip'][1] +
                target_keypoints['right_hip'][1]) / 2;

    (max_torso_yrange, max_torso_xrange,
      max_body_yrange, max_body_xrange) = determine_torso_and_body_range(
          keypoints, target_keypoints, center_y, center_x)

    crop_length_half = np.amax(
        [max_torso_xrange * 1.9, max_torso_yrange * 1.9,
          max_body_yrange * 1.2, max_body_xrange * 1.2])

    tmp = np.array(
        [center_x, image_width - center_x, center_y, image_height - center_y])
    crop_length_half = np.amin(
        [crop_length_half, np.amax(tmp)]);

    crop_corner = [center_y - crop_length_half, center_x - crop_length_half];

    if crop_length_half > max(image_width, image_height) / 2:
      return init_crop_region(image_height, image_width)
    else:
      crop_length = crop_length_half * 2;
      return {
        'y_min': crop_corner[0] / image_height,
        'x_min': crop_corner[1] / image_width,
        'y_max': (crop_corner[0] + crop_length) / image_height,
        'x_max': (crop_corner[1] + crop_length) / image_width,
        'height': (crop_corner[0] + crop_length) / image_height -
            crop_corner[0] / image_height,
        'width': (crop_corner[1] + crop_length) / image_width -
            crop_corner[1] / image_width
      }
  else:
    return init_crop_region(image_height, image_width)

def crop_and_resize(image, crop_region, crop_size):
  """Crops and resize the image to prepare for the model input."""
  boxes=[[crop_region['y_min'], crop_region['x_min'],
          crop_region['y_max'], crop_region['x_max']]]
  output_image = tf.image.crop_and_resize(
      image, box_indices=[0], boxes=boxes, crop_size=crop_size)
  return output_image

def run_inference(movenet, image, crop_region, crop_size):
  """Runs model inference on the cropped region.

  The function runs the model inference on the cropped region and updates the
  model output to the original image coordinate system.
  """
  image_height, image_width, _ = image.shape
  input_image = crop_and_resize(
    tf.expand_dims(image, axis=0), crop_region, crop_size=crop_size)
  # Run model inference.
  keypoints_with_scores = movenet(input_image)
  # Update the coordinates.
  for idx in range(17):
    keypoints_with_scores[0, 0, idx, 0] = (
        crop_region['y_min'] * image_height +
        crop_region['height'] * image_height *
        keypoints_with_scores[0, 0, idx, 0]) / image_height
    keypoints_with_scores[0, 0, idx, 1] = (
        crop_region['x_min'] * image_width +
        crop_region['width'] * image_width *
        keypoints_with_scores[0, 0, idx, 1]) / image_width
  return keypoints_with_scores

In [ ]:
# from IPython.display import display, HTML,clear_output
# from moviepy.editor import ImageSequenceClip
# def to_mp4(output_images, filename, fps=30):
#     clip = ImageSequenceClip([image[..., ::-1] for image in output_images], fps=fps)
#     clip.write_videofile(filename, codec='libx264')

# video_path = video_file
# cap = cv2.VideoCapture(video_path)
# num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# fps = cap.get(cv2.CAP_PROP_FPS)
# image_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# image_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

# crop_region = init_crop_region(image_height, image_width)
# cv2.namedWindow('Video with Subtitles', cv2.WINDOW_NORMAL)
# cv2.resizeWindow('Video with Subtitles', 800, 600)
# output_images = []
# bar = display(progress(0, num_frames-1), display_id=True)
# last_frame = 0
# frame_idx = 0
# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break
#     clear_output(wait=True)
#     print(frame_idx)

#     keypoints_with_scores = run_inference(
#             movenet, frame, crop_region,
#             crop_size=[input_size, input_size])
#     img = draw_prediction_on_image(
#             frame.astype(np.int32),
#             keypoints_with_scores, crop_region=None,
#             close_figure=True, output_image_height=300)

#         #output_images.append(img)

#         # Display the current image
#     cv2.imshow('Video with Subtitles', img)


#     crop_region = determine_crop_region(
#             keypoints_with_scores, image_height, image_width)
#     bar.update(progress(frame_idx, num_frames-1))
#     last_frame = frame_idx
#     key = cv2.waitKey(30)  # Adjust the delay as needed (milliseconds)

#     if key == 27:  # ESC key to exit
#         break
#     elif key == 83 or key == 100:
#         frame_idx += 1


# cap.release()
# cv2.destroyAllWindows()
# # Prepare MP4 visualization.
# #to_mp4(output_images, 'output.mp4', fps=fps)

#### Create CSV file

In [2]:

fields = ['id', 'angle', 'val', 'text']


rows = [['Nikhil', 'COE', '2', '9.0'],
        ['Sanchit', 'COE', '2', '9.1'],
        ['Aditya', 'IT', '2', '9.3'],
        ['Sagar', 'SE', '1', '9.5'],
        ['Prateek', 'MCE', '3', '7.8'],
        ['Sahil', 'EP', '2', '9.1']]

filename = "data.csv"

with open(filename, 'w') as csvfile:

    csvwriter = csv.writer(csvfile)

    csvwriter.writerow(fields)

    csvwriter.writerows(rows)


#### Process video with movenet

In [ ]:
from google.colab.patches import cv2_imshow

def to_mp4(output_images, filename, fps=30):
    clip = ImageSequenceClip([image[..., ::-1] for image in output_images], fps=fps)
    clip.write_videofile(filename, codec='libx264')

video_path = video_file



print("OpenCV-Python Version {}".format(cv2.__version__))

cap = cv2.VideoCapture(video_path)
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
image_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
image_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

crop_region = init_crop_region(image_height, image_width)
# cv2.namedWindow('Video with Subtitles', cv2.WINDOW_NORMAL)
# cv2.resizeWindow('Video with Subtitles', 800, 600)
length = int(cap. get(cv2. CAP_PROP_FRAME_COUNT))
print( length )
output_images = []
bar = display(progress(0, num_frames-1), display_id=True)
last_frame = 0
frame_idx = 0

try:
    while cap.isOpened():
        #cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if not ret:
            break
        clear_output(wait=True)
        print(frame_idx)
        print( length )
        #if last_frame!=frame_idx:
        keypoints_with_scores = run_inference(
                movenet, frame, crop_region,
                crop_size=[input_size, input_size])
        img = draw_prediction_on_image(
                frame.astype(np.int32),
                keypoints_with_scores, crop_region=None,
                close_figure=True, output_image_height=300)

        output_images.append(img)

            # Display the current image
        #cv2_imshow(img)
        crop_region = determine_crop_region(
                keypoints_with_scores, image_height, image_width)
        #bar.update(progress(frame_idx, num_frames-1))
        #last_frame = frame_idx
        #key = cv2.waitKey(30)  # Adjust the delay as needed (milliseconds)

        # if key == 27:  # ESC key to exit
        #     break
        # elif key == 83 or key == 100:
        frame_idx += 1
finally:
    cap.release()
    cv2.destroyAllWindows()
# Prepare MP4 visualization.
to_mp4(output_images, 'output.mp4', fps=fps)

8307
8308
Moviepy - Building video output.mp4.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4


### divide text into sentences and add punctuation with ml model

In [ ]:

text =''
for obj in subtitles:
    text+=obj['text']

print(len(text))
print(text)


from deepmultilingualpunctuation import PunctuationModel
model = PunctuationModel()

result = model.restore_punctuation(text)
print(len(result))



4705
we will approach the squat in two phasesfirst unloaded to solve problemsassociated with the bottom position andthen loaded to learn how to apply thebottom position to the hip drive usedfor heavier weights since the majorityof the problems with the squat happenedat the bottom this method expedites theprocess quite effectively we will use afairly neutral foot placement with theheels about shoulder width apart and thetoes pointed out at about 30 degreesmany people will assume a stance withtoes pointed too forward so you may needto point them out more than you want tonext you're going to assume the positionyou will be in at the bottom of a squatwithout the barsquat down all the way to a position inwhich the apex of the hip crease dropsjust below the top of the patella putyour elbows against your knees with thepalms of your hands together and shoveyour knees out notice your feet are flaton the floor your knees are shoved outto where they are in a parallel linewith your feet and just a 

C:\Users\Damian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


C:\Users\Damian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\pipelines\token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.NONE"` instead.
  warnings.warn(


4790


### removing artificial connections in words (auto generating subtitles from yt isn't ideal)
### also pos-tags are added here

In [ ]:
sents = nltk.sent_tokenize(result)

import wordsegment
from wordsegment import load, segment
load()
from nltk.tokenize import word_tokenize

# segment powoduje również tokenizacje zdania dlatego ten etap(tokenizacji) zostanie pominięty.
sents = [(segment(sent)) for sent in sents]
#porter = nltk.PorterStemmer()
#sents = [[porter.stem(t) for t in sent] for sent in sents]
sents = [nltk.pos_tag(sent) for sent in sents]
grammar = r"""
  NP: {<DT|PP\$>?<JJ>*<NN>}
      {<NNP>+}
"""
# grammar = r"""
#   NP: {<DT>?<JJ>*<NN>}
#   VP: {<VB.*><NP|PP>*}
#   PP: {<IN><NP>}
#   ADJP: {<JJ>}
#   ADVP: {<RB.*>}
# """
cp = nltk.RegexpParser(grammar)

# class ConsecutiveNPChunkTagger(nltk.TaggerI):

#     def __init__(self, train_sents):
#         train_set = []
#         for tagged_sent in train_sents:
#             untagged_sent = nltk.tag.untag(tagged_sent)
#             history = []
#             for i, (word, tag) in enumerate(tagged_sent):
#                 featureset = npchunk_features(untagged_sent, i, history)
#                 train_set.append( (featureset, tag) )
#                 history.append(tag)
#         self.classifier = nltk.MaxentClassifier.train(
#             train_set, algorithm='megam', trace=0)

#     def tag(self, sentence):
#         history = []
#         for i, word in enumerate(sentence):
#             featureset = npchunk_features(sentence, i, history)
#             tag = self.classifier.classify(featureset)
#             history.append(tag)
#         return zip(sentence, history)

# class ConsecutiveNPChunker(nltk.ChunkParserI):
#     def __init__(self, train_sents):
#         tagged_sents = [[((w,t),c) for (w,t,c) in
#                          nltk.chunk.tree2conlltags(sent)]
#                         for sent in train_sents]
#         self.tagger = ConsecutiveNPChunkTagger(tagged_sents)

#     def parse(self, sentence):
#         tagged_sents = self.tagger.tag(sentence)
#         conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
#         return nltk.chunk.conlltags2tree(conlltags)

# def npchunk_features(sentence, i, history):
#      word, pos = sentence[i]
#      return {"pos": pos}
# chunker = ConsecutiveNPChunker(train_sents)
# print(chunker.evaluate(test_sents))


# sents = [cp.parse(sent) for sent in sents]





In [ ]:
# nltk.download('maxent_ne_chunker')
# nltk.download('treebank')
# print(sents[30])
sent = nltk.corpus.treebank.tagged_sents()[22]
#print(sent)
print(sents[15])
#print(nltk.ne_chunk(sent))
print(nltk.ne_chunk(sents[25]))

# sentence = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"),
# ("dog", "NN"), ("barked", "VBD"), ("at", "IN"),  ("the", "DT"), ("cat", "NN")]

# grammar = "NP: {<DT>?<JJ>*<NN>}"

# cp = nltk.RegexpParser(grammar)
# result = cp.parse(sentence)
# print(result)
grammar = r"NP: {<[CDJNP].*>+}"
cp = nltk.RegexpParser(grammar)
# print(cp.evaluate(sents))
# result.draw()

[('measured', 'VBN'), ('from', 'IN'), ('the', 'DT'), ('markings', 'NNS'), ('placed', 'VBN'), ('on', 'IN'), ('the', 'DT'), ('bar', 'NN'), ('for', 'IN'), ('this', 'DT'), ('purpose', 'NN'), ('a', 'DT'), ('standard', 'JJ'), ('powerbar', 'NN'), ('has', 'VBZ'), ('16to17', 'CD'), ('inches', 'NNS'), ('between', 'IN'), ('the', 'DT'), ('ends', 'NNS'), ('of', 'IN'), ('the', 'DT'), ('inside', 'JJ'), ('neural', 'JJ'), ('and', 'CC'), ('32', 'CD'), ('inches', 'NNS'), ('between', 'IN'), ('the', 'DT'), ('finger', 'NN'), ('marks', 'NNS')]
(S
  again/RB
  heels/NNS
  should/MD
  be/VB
  about/IN
  shoulder/NN
  width/NNS
  apart/RB
  with/IN
  toes/NNS
  pointed/VBN
  out/RP
  about/IN
  30/CD
  degrees/NNS
  at/IN
  this/DT
  point/NN
  you/PRP
  are/VBP
  ready/JJ
  to/TO
  squat/VB
  with/IN
  the/DT
  empty/JJ
  bar/NN)


### Finding sentences with technique rules(unfinished) (regexp: noun(body part) and verb)

In [ ]:
text = "When performing squats with a barbell, ensure your back is straight, knees do not extend beyond your toes, and the barbell rests securely on your shoulders."

from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')
part = wn.synsets('body_part')[0]

def is_body_part(candidate):
    for ss in wn.synsets(candidate):
        # only get those where the synset matches exactly
        name = ss.name().split(".", 1)[0]
        if name != candidate:
            continue
        hit = part.lowest_common_hypernyms(ss)
        if hit and hit[0] == part:
            return True
    return False

# for word in sents[0]:
#     print(is_body_part(word[0]), word[0], sep="\t")

# Procesowanie każdego zdania
# for sentence in sents:
#     if any(is_body_part(t[0].lower()) for t in sentence):
#         print(f"Zdanie zawiera część ciała: {sentence}")

import nltk
from nltk import CFG
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from nltk.parse import ChartParser

# Lista części ciała
body_parts = ["head", "arm", "leg", "hand", "foot", "eye", "ear", "nose", "mouth", "shoulder", "knee", "elbow"]

# Definicja gramatyki bezkontekstowej z użyciem POS tags
grammar = CFG.fromstring("""
  S -> NP VP
  NP -> DT JJNN | JJNN
  VP -> VBZ NP | VBZ ADJP | VBZ PP
  DT -> 'the' | 'a' | 'his' | 'her'
  JJNN -> JJ NN | JJNN JJ NN
  JJ -> 'badly' | 'quickly' | 'slowly' | 'fast'
  NN -> 'head' | 'arm' | 'leg' | 'hand' | 'foot' | 'eye' | 'ear' | 'nose' | 'mouth' | 'shoulder' | 'knee' | 'elbow'
  VBZ -> VB
  ADJP -> JJ NP
  NP -> DT JJNN
  PP -> IN NP
  IN -> 'in'
""")

# Tworzenie parsera
parser = ChartParser(grammar)

# Tokenizacja tekstu na zdania
for sentence in sents:
    # Sprawdzanie czy zdanie pasuje do gramatyki
    #print(sentence)
    words = [word for word, tag in sentence]
    try:
        for tree in parser.parse(words):
            # # Sprawdzanie czy pierwsza fraza rzeczownikowa jest częścią ciała
            # np = tree[0]
            # if np.label() == 'NP' and np[0][0].lower() in body_parts:
            #     print(f"Zdanie zawiera część ciała jako podmiot: {sentence}")
            tree.pretty_print()
    except ValueError:
            # Jeżeli parser nie znajdzie pasującego drzewa, przechodzi do następnego zdania
        #print("nie ma drzewa")
        continue
import re

#pattern = r'(head|arm|leg|hand|foot|eye|ear|nose|back|mouth|shoulder|knee|elbow)'
pattern = r'\b(head|arm|leg|hand|foot|eye|ear|nose|back|mouth|shoulder|knee|elbow)\b\s+(\w+)'

sents_for_regexp = nltk.sent_tokenize(result)
sents_for_regexp = [(segment(sent)) for sent in sents_for_regexp]
for sentence in sents_for_regexp:
    #print(sentence)
    sem = ' '.join([str(elem) for elem in sentence])
    #print(sem)
    match = re.search(pattern,sem)
    if match:
        print(sem)
        #print(match.string)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Damian\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


we will use a fairly neutral foot placement with the heels about shoulder width apart and the toes pointed out at about 30 degrees many people will assume a stance with toes pointed too forward so you may need to point them out more than you want
your back should be as flat as you can get it
also notice that your back is inclined at about a45 degree angle not at all vertical and your eyes are looking down at the floor a few feet in front of you
this movement keeps your weight solidly over the whole foot instead of letting it shift to the toes
grip width for the squat will vary with shoulder width and flexibility but in general the hands will be between these two markings
the elbows should be lifted up to trap the bar between the hands and the back elbows should be up but not high
with your grip in place and your hands and thumbs on top of the bar dip your head under the bar and come up into position with the bar on your back just below the spine of the scapula the bone you feel at the 

In [ ]:
for sent in sents:
    for tuple in sent:
        print(tuple[0],end = " ")
    print("\n")

we will approach the squat in two phases first unloaded to solve problems associated with the bottom position and then loaded to learn how to apply the bottom position to the hip drive used for heavier weights 

since the majority of the problems with the squat happened at the bottom this method expedites the process quite effectively 

we will use a fairly neutral foot placement with the heels about shoulder width apart and the toes pointed out at about 30 degrees many people will assume a stance with toes pointed too forward so you may need to point them out more than you want 

to next your e going to assume the position you will be in at the bottom of a squat without the bar squat down all the way to a position in which the apex of the hip crease drops just below the top of the patella 

put your elbows against your knees with the palms of your hands together and shove your knees out 

notice your feet are flat on the floor 

your knees are shoved out to where they are in a paralle